In [ ]:
import torch
import torchvision
import sys
sys.path.append('../')
from config import *
from PE_helper import *
from pano_mask_eval_helper import *
from tqdm import tqdm
from model import *
from file_helper import *

In [3]:
# DataLoader
import torchvision.transforms as T
from torchvision import transforms
from torchvision.transforms import ToTensor
from PIL import Image
import json
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

transform = T.Compose([    
    transforms.Resize((512, 1024)),
    T.ToTensor(),  # convert PIL image to PyTorch tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize image
])
transform_no_norm = T.Compose([    
    T.ToTensor(),  # convert PIL image to PyTorch tensor
    transforms.Resize((512, 1024)),    
])
transform_norm= transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def uv_to_xyxy(u,v ):
    result = torch.zeros_like(v)
    result[:,0] = u[:,0] # top left x
    result[:,1] = v[:,0]
    result[:,2] = u[:,1] # right bottom x
    result[:,3] = v[:,3]
    return result

def debug_draw_bbox(img , bbox , normalize = True ):    
    h,w = img.shape[-2:]
    debug_img = img.permute(1,2,0).detach().cpu().numpy()
    debug_img = np.ascontiguousarray(debug_img)

    boxes = bbox.reshape(-1,4)
    for box in boxes:
        if(normalize):
            p0 = np.int32([box[0] * w , box[1] * h])
            p1 = np.int32([box[2] * w , box[3] * h])
        else:
            p0 = np.int32([box[0]  , box[1] ])
            p1 = np.int32([box[2]  , box[3] ])
        debug_img = cv2.rectangle(debug_img, tuple(p0) ,  tuple(p1) , (0,255,0) , 2 )
    return debug_img

def collate_fn(data):    
    tensors, targets = zip(*data)    
    return tensors, targets

class ZillowDataset(torch.utils.data.Dataset):
    def __init__(self, transforms=None , anno_path="" , mode ="train" , device = 'cuda'):        
        self.transforms = transforms
        self.mode= mode
        self.annos = []
        self.device = device

        with open(anno_path  , 'r') as f:
            self.annos =  json.load(f)
            print("anno loaded " , len(self.annos))        
        
        

    def __getitem__(self, idx):
        # TODO: Load your data here        
        img_path = os.path.join(ZILLOW_DATASET_FOLDER , self.annos[idx]['image'] )                
        img = Image.open(img_path)

        if self.transforms is not None:            
            img = self.transforms(img)        

        h , w = img.shape[-2:]
        u = torch.as_tensor(self.annos[idx]['u']) *w
        v = torch.as_tensor(self.annos[idx]['sticks_v'])*h

        target = {}
        bboxes = uv_to_xyxy(  u , v)        
        target['boxes'] = bboxes.to(self.device)
        target['labels'] = torch.ones(len(self.annos[idx]['u'])).view(-1).to(torch.int64) .to(self.device)
        
        # [Debug: ]
        '''
        for box in bboxes:
            debug_img = debug_draw_bbox(img , box)
            plt.imshow(debug_img)
            plt.show()
        '''

        #return img, target
        return img.to(self.device) , target

    def __len__(self):        
        return len(self.annos)

def anno_to_list(anno_b):
    anno_list =[]
    for anno in anno_b:
        anno_list.append({"boxes": anno["boxes"] , "labels" : anno['labels']})
    return anno_list

dataset_train = ZillowDataset(transforms=transform , anno_path= '../anno/train_visiable_20_no_cross.json' )
dataset_test = ZillowDataset(transforms=transform , anno_path= '../anno/test_visiable_10_no_cross.json' )
dataset_test_no_norm = ZillowDataset(transforms=transform_no_norm , anno_path= '../anno/test_visiable_10_no_cross.json' )
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4, shuffle=True, num_workers=0  , collate_fn = collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=4, shuffle=False, num_workers=0  , collate_fn = collate_fn)
data_loader_test_no_norm = torch.utils.data.DataLoader(dataset_test_no_norm, batch_size=2, shuffle=False, num_workers=0  , collate_fn = collate_fn)


#=========================================
#               Setting 
#=========================================
MAX_TRAIN_EPOCHES = 50
RUN_NAME = 'SMALL_TEST_200'

model_2cls = model_2cls.to('cuda')
optimizer = optim.Adam(model_2cls.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
writer = SummaryWriter(RUN_NAME)



def loss_fn(output):
    losses = sum(loss for loss in output.values())
    return losses
def inf_loss_fn(output):
    labels = []
    boxes = []
    for b_out in output:
        label = b_out['labels']
        box = b_out['boxes']

        labels.append(label)
        boxes.append(box)
    labels = torch.cat(labels)
    boxes = torch.cat(boxes)

@torch.no_grad()
def write_loss(output , type='train' , step=0 ):
    for key, value in output.items():
        writer.add_scalar(f"{type}/{key}" , value , step)
    pass

@torch.no_grad()
def inf_fn(model , data_loader , vis_data , epoch , log_folder= ""):    
    model.train()    
    
    cnt = 0
    '''
    '''
    for data in tqdm(data_loader):
        img_b , anno_b = data
        anno_b_list = anno_to_list(anno_b)        
        output = model(img_b , anno_b_list)
        write_loss(output , 'test' , epoch * len(data_loader) + cnt)           
        cnt+=1

    # visualize
    model.eval()        
    for i in tqdm(range(5)):
        img , anno      = vis_data[i]                
        img_b_norm      = transform_norm(img).unsqueeze(0)
        
        output_b        = model(img_b_norm)
        
        for j , out in enumerate(output_b):
            box = out['boxes'].detach().cpu().numpy()                        
            debug_img = debug_draw_bbox(img, box , normalize=False)

            gt_box = anno['boxes'].detach().cpu().numpy()                    
            debug_img_gt =  debug_draw_bbox(img, gt_box , normalize=False)

            fig, axs = plt.subplots(1, 2 ,  figsize=(20, 5))                                     
            # Plot the images
            axs[0].imshow(debug_img)
            axs[0].axis("off")
            axs[1].imshow(debug_img_gt)
            axs[1].axis("off")
            
            plt.savefig(os.path.join(log_folder , f"vis_test_ep{epoch}-it{i}.jpg") , dpi = 150)
            plt.close(fig)
        pass
    pass

# Test Train
def train_fn(model,optimizer  , epoch , data_loader , log_folder="" ):
    cnt = 0
    for data in tqdm(data_loader):
        model.train()
        img_b , anno_b = data
        anno_b_list = anno_to_list(anno_b)       

        output = model(img_b , anno_b_list)
        loss = loss_fn(output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    

        write_loss(output , 'train' , epoch * len(data_loader) + cnt)
        cnt+=1
            #break

        # [ Test ]
        '''
        '''
    if (epoch % 5 ==0):
        with torch.no_grad():
            model.eval()
            output_b        = model(img_b)
            debug_cnt = 0
            for img , anno , out in zip(img_b , anno_b , output_b):        
                pred_box = out['boxes'].detach().cpu().numpy()                        
                debug_img = debug_draw_bbox(img, pred_box , normalize=False)

                gt_bbox = anno['boxes']        
                debug_img_gt = debug_draw_bbox(img, gt_bbox.detach().cpu().numpy() ,  normalize=False)
                fig, axs = plt.subplots(1, 2 ,  figsize=(20, 5))                                     
                # Plot the images
                axs[0].imshow(debug_img)
                axs[0].axis("off")
                axs[1].imshow(debug_img_gt)
                axs[1].axis("off")
                
                plt.savefig(os.path.join(log_folder , f"vis_train_ep{epoch}-it{debug_cnt}.jpg") , dpi = 150)
                plt.close(fig)

                debug_cnt+=1
    return model


train_eppch = 0
eval_eppch = 0
log_folder = create_folder( os.path.join(os.getcwd() , "output" , RUN_NAME) )
eval_helper = PR_Eval_Helper(writer=writer , log_folder=log_folder)

for epoch in range(1,MAX_TRAIN_EPOCHES+1):
    '''
    '''
    train_fn(model_2cls , optimizer, train_eppch , data_loader_train , log_folder)
    train_eppch+=1

    if(epoch % 5 ==0):
        inf_fn(model_2cls , data_loader_test , dataset_test_no_norm , eval_eppch , log_folder)
        eval_eppch+=1
    #inf_fn(model_2cls , data_loader_test , dataset_test_no_norm , eval_eppch , log_folder)
    #break

anno_b ({'boxes': tensor([[382.3250, 231.7050, 400.2300, 319.5150],
        [772.8050, 236.9150, 832.6850, 301.8500]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')}, {'boxes': tensor([[414.1000, 116.3100, 596.7350, 446.9100]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[172.3900, 237.7600, 189.7400, 306.1000],
        [300.3800, 226.4050, 330.8050, 322.4400],
        [445.3300, 143.3050, 590.5900, 449.6150]], device='cuda:0'), 'labels': tensor([1, 1, 1], device='cuda:0')}, {'boxes': tensor([[229.8600, 140.3200, 736.6900, 412.0600]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')})


  0%|          | 0/5 [00:00<?, ?it/s]

anno_b ({'boxes': tensor([[411.8400, 127.2550, 584.7500, 448.9150]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[376.8700, 147.4800, 541.2250, 431.3950]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[414.1000, 116.3100, 596.7350, 446.9100]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[430.5100, 195.2650, 507.7150, 371.3350],
        [799.7850, 170.9350, 958.9600, 443.4000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')})


 20%|██        | 1/5 [00:00<00:02,  1.38it/s]

anno_b ({'boxes': tensor([[530.5950, 236.6900, 545.2800, 302.2250]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[233.6150, 114.4700, 590.1850, 490.9700]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[240.1350, 227.0250, 273.1500, 336.3200],
        [454.5550, 220.2200, 482.2150, 326.8100],
        [513.6250, 222.8750, 546.7900, 340.8150],
        [651.3100, 211.2200, 690.8150, 340.3150]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1], device='cuda:0')}, {'boxes': tensor([[200.4750, 119.4850, 398.8600, 450.9450]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')})


...

 60%|██████    | 3/5 [00:02<00:01,  1.37it/s]

anno_b ({'boxes': tensor([[229.8600, 140.3200, 736.6900, 412.0600]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[430.5100, 195.2650, 507.7150, 371.3350],
        [799.7850, 170.9350, 958.9600, 443.4000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')}, {'boxes': tensor([[200.4750, 119.4850, 398.8600, 450.9450]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[391.7450, 101.8250, 636.6100, 463.1600],
        [768.1450, 208.6150, 813.9150, 354.0300]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')})


 80%|████████  | 4/5 [00:02<00:00,  1.36it/s]

anno_b ({'boxes': tensor([[530.5950, 236.6900, 545.2800, 302.2250]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[133.2650, 114.7400, 337.6200, 458.9300]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[414.1000, 116.3100, 596.7350, 446.9100]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}, {'boxes': tensor([[411.8400, 127.2550, 584.7500, 448.9150]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')})


  0%|          | 0/5 [00:00<?, ?it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
 20%|██        | 1/5 [00:00<00:01,  3.21it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
 40%|████      | 2/5 [00:00<00:00,  3.20it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
 80%|████████  | 4/5 [00:01<00:00,  3.26it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB